# First Attention Blocks
<img src="image_assets/basic_attention.png" alt="drawing" width="600"/>

## Imports

In [1]:
# Standard Lib
import os
import math
import random
from time import time
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt

# Tokenization
import spacy 

# Loading Bar
from tqdm import tqdm

# Torch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Dataloader Custom Module
from sample_dataloader import get_dataloaders

## Constants

In [3]:
data_root = os.path.join(Path(os.getcwd()).parent.parent.parent, "Datasets/")
gpu = torch.device("cuda:0")

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!python -m spacy download de_core_news_sm

## Datasets

In [4]:
trainset, validset, testset, de_vocab, en_vocab = get_dataloaders(batch_size=128, device=gpu, data_root=data_root)

/opt/conda/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/conda/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/opt/conda/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information

In [6]:
# all these indices are the same for french and english
PAD_IDX = de_vocab['<pad>']
SOS_IDX = de_vocab['<sos>']
EOS_IDX = de_vocab['<eos>']

In [13]:
vals = next(iter(trainset))
vals.src.unsqueeze(0).shape, vals.src.shape

(torch.Size([1, 30, 128]), torch.Size([30, 128]))

## Attention
In **Transformers 1 - Before Attention** each forward pass went as 
```python
y, s = '<s>', encoder(x)
for _ in range(N):
    y, s = decoder(y, s)
```

Now we introduce **Attention**
```python
# Encoding Stage (inp_vec (x) -> hidden_states)
hidden_states, s = encoder(x)

# Decoding Stage(s)

# 1) computing the attention weights
alignment_scores = []
for h_i in hidden_states:
    e_i = f_att(s, h_i)
    alignment_scores.append(e_i)
    
attention_weights = F.softmax(alignment_scores)

# 2) computing the context vector
c = torch.zeros(1, hidden_dim)
for a_i, h_i in zip(attention_weights, hidden_states):
    c += a_i * h_i
    
# 3) decoding
y = '<s>'
y, s = decoder(y, c)
```


## Implementation Details
### Encoder Parameter Measurements
- Input is a () matrix
- Output is a () matrix

### Attention Parameter Measurements
- Input is a () matrix
- Output is a () matrix

### Decoder Parameter Measurements
- Input is a () matrix
- Output is a () matrix

In [14]:
class Encoder(nn.Module):
    def __init__(self, inp_vocab_len, num_features, num_hidden, num_layers, dropout):
        """
        Inputs:
            (encoder lstm params)
                num_hidden_features: The number of features in the hidden state h
                num_encoder_layers: The number of layers in the lstm
                dropout: the probability of dropout

            (embedding params)
                inp_vocab_len: The size of the dictionary of embeddings
                num_features: The size of each embedding vector

            (regularization)
                dropout
        """
        super().__init__()
        
        self.embedding = nn.Embedding(num_embeddings=inp_vocab_len, 
                                      embedding_dim=num_features)
        self.dropout = nn.Dropout(dropout)
        
        self.lstm = nn.LSTM(input_size=num_features, 
                            hidden_size=num_hidden, 
                            num_layers=num_layers,
                            dropout=dropout)
    
    def forward(self, x):
        """
        Input:
            (x) 
                an encoded german sentence
                shape: [1, (number of tokens in batch), (batch size)]
        
        Output:
            (hidden_states)
                tensor containing the output features (h_t) from the last layer of the LSTM, for each t
            (h_t)
                tensor containing the hidden state for t = seq_len
            (c_t)
                tensor containing the cell state for t = seq_len.
        """
        embedding = self.dropout(self.embedding(x))
        hidden_states, (h_t, c_t) = self.lstm(embedding)
        return hidden_states, (h_t, c_t)

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.f_att = nn.Linear()
    
    def forward(self, s, encoder_hidden_states):
        """
        s : current decoder hidden state
        encoder hidden states: : tensor containing the output features (h_t) from the last layer of the LSTM, for each t
        """
        
        # compute alignment scores
        alignment_scores = self.f_att(torch.cat((s, encoder_hidden_states), dim=2))
        
        # compute attention weights
        return F.softmax(alignment_scores)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, attention):
        super().__init__()
        # English Embeddings
        self.embedding = nn.Embedding()
        self.dropout = nn.Dropout()
        
        # attention block
        self.attention = attenion
        
        # Decoder
        self.lstm = nn.LSTM()
        self.fc_out = nn.Linear()
    
    def forward(self, x, s, cell, encoder_outputs):
        """
        x: the previous token
        s: current decoder hidden state
        cell: current cell state
        encoder outputs: tensor containing the output features (h_t) from the last layer of the LSTM, for each t
        """
        embedding = self.dropout(self.embedding(x))
        
        # attention
        attention_weights = self.attention(s, encoder_outputs)
        context_vector = torch.bmm(weights, hidden)
        
        # decoding
        output, (s, cell) = self.lstm(context_vector, cell)
        
        prediction = self.fc_out(prediction)
        return prediction, s, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init()
    
    def forward(self, x):
        outputs = torch.zeros(trg_len, batch_size, self.trg_vocab_size).to(self.device)
        
        encoder_outputs, s, c_t = self.encoder(src)
        
        # this shoud just be replaced with <s> or whatever our start token is
        x = trg[0,:]
        
        for t in range(1, trg_len):
            prediction, s, cell = self.decoder(x, s, cell, encoder_outputs)
            
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(1) 
    
            x = trg[t] if teacher_force else top1
        
        return outputs

### Initialization + Number of Params

In [ ]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 256
DEC_HID_DIM = 256
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, gpu, OUTPUT_DIM).to(gpu)

In [ ]:
# initialize model weights
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

### Training

In [ ]:
def train(model, iterator, optimizer, criterion, clip=1, num_epochs=10):
    model.train()
    losses = []
    for _ in range(num_epochs):
        for i, batch in tqdm(enumerate(iterator), desc="iteration"):
            src = batch.src
            trg = batch.trg 
            
            optimizer.zero_grad()

            output = model(src, trg)

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            losses.append(loss.item())
            loss.backward()

            # clip the gradients to prevent them from exploding (a common issue in RNNs)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

            optimizer.step()
                    
    return model, losses

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
model, losses = train(model, trainset, optimizer, criterion)

### Training

In [ ]:
plt.scatter(x=list(range(len(losses))), y=losses)

### Testing

In [ ]:
def tensor_2_str(tensor, vocab=de_vocab.itos):
    return " ".join([vocab[int(token)] for token in tensor if vocab[int(token)] not in ['<eos>', '<pad>', '.']])

In [ ]:
with torch.no_grad():
    sample = next(iter(trainset))
    src, trg = sample.src, sample.trg
    output = model(src, trg)
    output_tensor = output.argmax(2)[:, 0]
    target_tensor = trg[:, 0]
    
    output = tensor_2_str(output_tensor, en_vocab.itos)
    expected = tensor_2_str(target_tensor, en_vocab.itos)
    N = max(len(output), len(expected)) + len("Expected: ")
    
    print("="*N)
    print("Output: {}".format(output).center(N))
    print("="*N)
    
    print("="*N)
    print("Expected: {}".format(expected).center(N))
    print("="*N)